# BorutaPy 

# 1. 데이터 구조 보기

1-1. 데이터 읽어 DataFrame으로 변경

In [1]:
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from sklearn.preprocessing import MinMaxScaler
import sklearn
# Matplotlib에서 마이너스 기호가 제대로 보이도록 설정
plt.rc('axes', unicode_minus=False)


In [2]:
df = pd.read_csv('cancer_calssification.csv')#5주차에서 제작된 dataframe 가져오기
df

,ELMO2,CREB3L1,RPS11,PNMA1,MMP2,C10orf90,ZHX3,ERCC5,GPR98,RXFP3,...,GRIP2,GPLD1,RAB8A,RXFP2,PIK3IP1,SLC39A6,SNRPD2,AQP7,CTSC,label
0,0.436333,-0.04350,0.208125,0.23975,-1.130167,-2.20475,1.177167,0.40650,0.256500,0.5150,...,-0.29200,0.089500,0.1273,0.3170,2.26575,3.0265,-0.2115,-0.9400,-0.0443,1
1,1.259750,0.92025,0.347875,0.50300,0.208333,-1.79025,-0.637167,0.10025,1.095750,0.3545,...,-0.15900,0.039500,0.6641,0.2040,2.38350,2.8252,0.2634,-0.5105,-0.0601,1
2,0.673667,1.11950,0.214875,0.67975,0.520167,-2.25500,-0.302833,-0.84175,0.477500,0.7090,...,-0.13500,-0.784000,-0.2225,0.0005,2.61775,-1.0733,0.1832,-0.6545,0.9376,1
3,-0.290000,-0.35450,0.668625,-0.37025,-0.692167,0.48175,-0.986000,0.30525,-1.679625,0.4615,...,-0.25975,1.022667,0.2466,0.3655,1.08600,-2.1321,0.3481,-0.7510,0.3642,1
4,0.602917,-0.24275,0.313875,0.34500,-0.565000,-2.45650,0.146000,0.12600,0.772500,0.4125,...,-1.41775,-0.538167,0.8628,0.4150,2.86250,3.0570,0.1072,-1.0535,-1.8482,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
585,-0.569083,0.53450,1.756000,0.25550,-0.468167,-1.24625,0.145333,-0.39975,0.069375,0.7115,...,-0.31125,-0.221000,-0.4884,0.2070,2.20125,-0.5748,-0.1637,2.7450,-0.9557,0
586,0.620333,1.04875,0.246500,0.15675,0.139333,-2.23225,0.291667,-0.66225,0.252875,0.5070,...,-0.07825,-0.900833,0.3608,0.6915,1.27225,0.9847,0.6462,-0.8920,-0.8784,1
587,1.031250,1.44600,0.060250,0.43075,-0.727167,-2.21400,0.291833,-0.02575,-1.342250,0.5720,...,-0.33450,-0.345167,0.3869,0.3250,1.67325,4.6337,-0.1753,-0.5270,-1.5026,1
588,1.157750,1.89825,0.352250,0.50025,2.484000,-1.91425,-0.458167,0.90300,0.034125,0.2410,...,0.34750,-1.205500,2.1037,0.4495,3.24200,-1.2651,-1.0757,1.2035,-0.0060,1


In [3]:
df_checknull = pd.DataFrame(df.isnull().sum())#각 열의 null 개수 데이터프레임
df_havenull = df_checknull[df_checknull[0]!=0]# null 값이 있는 열 데이터프레임
havenull_columns = list(df_havenull.index)# null 값이 있는 열의 이름 리스트
df_havenull#확인

,0
OR2K2,5
ADAM5P,14
KIAA1486,1
C9orf11,2
C8G,1
...,...
OR1J4,14
MLNR,1
SALL3,2
CCDC73,1


In [4]:
for col in havenull_columns:#null 값이 있는 열들에 대해서
    df.loc[(df[col].isnull()) & (df['label']==1),col] = df[df['label']==1][col].mean()
    #label이 1이면 label이 1인 행들 중 해당 열의 평균값으로 대체
    df.loc[(df[col].isnull()) & (df['label']==0),col] = df[df['label']==0][col].mean()
    #label이 0이면 label이 0인 행들 중 해당 열의 평균값으로 대체

In [5]:
df.isnull().values.any()#null이 모두 제거되었는 지 확인

False

In [6]:
# 정규화
scaler = MinMaxScaler()
scaler.fit(df.iloc[:, :-1])
data_scaled = scaler.transform(df.iloc[:, :-1])
data = pd.DataFrame(data=data_scaled, columns=df.iloc[:, :-1].columns)
data

,ELMO2,CREB3L1,RPS11,PNMA1,MMP2,C10orf90,ZHX3,ERCC5,GPR98,RXFP3,...,SLC2A11,GRIP2,GPLD1,RAB8A,RXFP2,PIK3IP1,SLC39A6,SNRPD2,AQP7,CTSC
0,0.466448,0.295543,0.347814,0.557454,0.188942,0.371352,0.940468,0.555387,0.590449,0.335814,...,0.691646,0.537521,0.511150,0.483807,0.419515,0.636201,0.795813,0.383254,0.195014,0.466619
1,0.644689,0.503561,0.387204,0.632098,0.407997,0.425897,0.355042,0.466696,0.752203,0.304386,...,0.488289,0.578175,0.502023,0.610426,0.404514,0.661055,0.772046,0.524509,0.247759,0.463121
2,0.517822,0.546568,0.349716,0.682215,0.459031,0.364740,0.462920,0.193889,0.633043,0.373801,...,0.260530,0.585511,0.351708,0.401297,0.377498,0.710501,0.311758,0.500654,0.230075,0.684042
3,0.309221,0.228416,0.477610,0.384490,0.260624,0.724874,0.242485,0.526064,0.217286,0.325338,...,0.423636,0.547379,0.681482,0.511947,0.425954,0.387177,0.186748,0.549703,0.218224,0.557074
4,0.502507,0.252536,0.377620,0.587297,0.281436,0.338224,0.607744,0.474153,0.689901,0.315743,...,0.478566,0.193413,0.396581,0.657294,0.432526,0.762164,0.799414,0.478049,0.181076,0.067182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
585,0.248809,0.420300,0.784096,0.561920,0.297283,0.497483,0.607529,0.321894,0.554383,0.374290,...,0.414473,0.531637,0.454474,0.338578,0.404912,0.622586,0.370615,0.397472,0.647550,0.264808
586,0.506278,0.531297,0.358630,0.533919,0.396705,0.367734,0.654746,0.245873,0.589750,0.334247,...,0.322566,0.602858,0.330382,0.538884,0.469233,0.426491,0.554742,0.638370,0.200909,0.281925
587,0.595227,0.617041,0.306134,0.611611,0.254896,0.370135,0.654800,0.430206,0.282311,0.346975,...,0.425600,0.524530,0.431809,0.545041,0.420577,0.511135,0.985572,0.394021,0.245733,0.143708
588,0.622610,0.714656,0.388437,0.631318,0.780427,0.409580,0.412799,0.699175,0.547589,0.282162,...,0.512692,0.732997,0.274771,0.949994,0.437106,0.842269,0.289113,0.126205,0.458246,0.475100


In [7]:
import seaborn as sns
import matplotlib.pyplot as plt
#corr_matrix = data.corr()
#print(corr_matrix)

In [8]:
new_df = pd.concat([df.loc[:, 'label'], data], axis=1)#정규화한 feature와 label 합치기
new_df#확인

,label,ELMO2,CREB3L1,RPS11,PNMA1,MMP2,C10orf90,ZHX3,ERCC5,GPR98,...,SLC2A11,GRIP2,GPLD1,RAB8A,RXFP2,PIK3IP1,SLC39A6,SNRPD2,AQP7,CTSC
0,1,0.466448,0.295543,0.347814,0.557454,0.188942,0.371352,0.940468,0.555387,0.590449,...,0.691646,0.537521,0.511150,0.483807,0.419515,0.636201,0.795813,0.383254,0.195014,0.466619
1,1,0.644689,0.503561,0.387204,0.632098,0.407997,0.425897,0.355042,0.466696,0.752203,...,0.488289,0.578175,0.502023,0.610426,0.404514,0.661055,0.772046,0.524509,0.247759,0.463121
2,1,0.517822,0.546568,0.349716,0.682215,0.459031,0.364740,0.462920,0.193889,0.633043,...,0.260530,0.585511,0.351708,0.401297,0.377498,0.710501,0.311758,0.500654,0.230075,0.684042
3,1,0.309221,0.228416,0.477610,0.384490,0.260624,0.724874,0.242485,0.526064,0.217286,...,0.423636,0.547379,0.681482,0.511947,0.425954,0.387177,0.186748,0.549703,0.218224,0.557074
4,1,0.502507,0.252536,0.377620,0.587297,0.281436,0.338224,0.607744,0.474153,0.689901,...,0.478566,0.193413,0.396581,0.657294,0.432526,0.762164,0.799414,0.478049,0.181076,0.067182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
585,0,0.248809,0.420300,0.784096,0.561920,0.297283,0.497483,0.607529,0.321894,0.554383,...,0.414473,0.531637,0.454474,0.338578,0.404912,0.622586,0.370615,0.397472,0.647550,0.264808
586,1,0.506278,0.531297,0.358630,0.533919,0.396705,0.367734,0.654746,0.245873,0.589750,...,0.322566,0.602858,0.330382,0.538884,0.469233,0.426491,0.554742,0.638370,0.200909,0.281925
587,1,0.595227,0.617041,0.306134,0.611611,0.254896,0.370135,0.654800,0.430206,0.282311,...,0.425600,0.524530,0.431809,0.545041,0.420577,0.511135,0.985572,0.394021,0.245733,0.143708
588,1,0.622610,0.714656,0.388437,0.631318,0.780427,0.409580,0.412799,0.699175,0.547589,...,0.512692,0.732997,0.274771,0.949994,0.437106,0.842269,0.289113,0.126205,0.458246,0.475100


In [9]:
#df.to_csv("cancer_calssification_scaled_data.csv",index=False)
#정규화된 dataframe 저장 이후 프로그램에서는 이를 불러와 사용 가능

In [10]:
rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)
boruta = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=42)#BurutaPy 생성

stats=data.iloc[:, :-1].describe()#정규화 한 값 분석
stats

,ELMO2,CREB3L1,RPS11,PNMA1,MMP2,C10orf90,ZHX3,ERCC5,GPR98,RXFP3,...,CSF2,SLC2A11,GRIP2,GPLD1,RAB8A,RXFP2,PIK3IP1,SLC39A6,SNRPD2,AQP7
count,590.000000,590.000000,590.000000,590.000000,590.000000,590.000000,590.000000,590.000000,590.000000,590.000000,...,590.000000,590.000000,590.000000,590.000000,590.000000,590.000000,590.000000,590.000000,590.000000,590.000000
mean,0.428189,0.484832,0.416739,0.504592,0.435925,0.460188,0.496585,0.460137,0.463221,0.321688,...,0.354978,0.484400,0.552295,0.397002,0.510299,0.423054,0.597244,0.508786,0.455692,0.325799
std,0.120375,0.179342,0.144956,0.174242,0.158249,0.142219,0.152620,0.143192,0.217094,0.080607,...,0.150480,0.129191,0.104119,0.158168,0.127614,0.091097,0.144747,0.212208,0.132049,0.183900
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.347013,0.351595,0.318853,0.388637,0.325275,0.372019,0.409707,0.366511,0.269928,0.276116,...,0.254004,0.397854,0.500459,0.288400,0.432987,0.376784,0.517361,0.344543,0.363913,0.205146
50%,0.427574,0.485458,0.408096,0.519848,0.445038,0.416669,0.493439,0.462714,0.499043,0.317750,...,0.328223,0.485905,0.556855,0.380807,0.511133,0.420511,0.610580,0.493052,0.447739,0.261390
75%,0.497858,0.612022,0.501964,0.620702,0.550447,0.490509,0.590643,0.562409,0.623997,0.356839,...,0.424206,0.554766,0.613232,0.490546,0.593307,0.471507,0.700369,0.666107,0.527052,0.393866
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# BurutaPy 실행 및 분석

In [11]:
new_df['label'].value_counts()

1    529
0     61
Name: label, dtype: int64

In [12]:
X = new_df.iloc[:, :-1].values
y = new_df.loc[:, 'label']
#X, y값 정하기
# borutaPy 동작
boruta.fit(X, y)
#selected_features = df.columns[:-1][boruta.support_]


Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	17814
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	17814
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	17814
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	17814
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	17814
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	17814
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	17814
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	0
Tentative: 	2079
Rejected: 	15735
Iteration: 	9 / 100
Confirmed: 	348
Tentative: 	1731
Rejected: 	15735
Iteration: 	10 / 100
Confirmed: 	348
Tentative: 	1731
Rejected: 	15735
Iteration: 	11 / 100
Confirmed: 	348
Tentative: 	1731
Rejected: 	15735
Iteration: 	12 / 100
Confirmed: 	373
Tentative: 	1205
Rejected: 	16236
Iteration: 	13 / 100
Confirmed: 	373
Tentative: 	1205
Rejected: 	16236
Iteration: 	14 / 100
Confirmed: 	373
Tentative: 	1205
Rejected: 	16236
Iteration: 	15 / 100
Confirmed: 	373
Tentative: 	120

BorutaPy(estimator=RandomForestClassifier(class_weight='balanced', max_depth=5,
                                          n_estimators=683, n_jobs=-1,
                                          random_state=RandomState(MT19937) at 0x278B0FE7940),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x278B0FE7940, verbose=2)

In [1]:
# 결과 출력
print(boruta.support_)#True=유의미/False=무의미
print(boruta.ranking_)#작을수록 의미
print(boruta.n_features_)#유의미한 feature 개수
selected_features = data.columns[boruta.support_]#유의미한 feature 이름
print(selected_features[:])#확인

NameError: name 'boruta' is not defined

In [ ]:
selected_df = new_df[selected_features]
selected_df = pd.concat([selected_df,new_df.loc[:, 'label']], axis=1)#정규화한 feature와 label 합치기
selected_df

In [ ]:
selected_df.to_csv('selected_features_data_1_3.csv', index=False)